---
# 올리브영 에센스/세럼/앰플 제품군 크롤링
---

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd

# 전체 페이지

In [54]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service,options=options)
time.sleep(0.2)

lastpage = 24
prd_items = []
prd_brands = []
prd_names = []
prd_prices = []
prd_infos = []
prd_thumbs = [] 
try: 
    for page in range(21, lastpage+1):
        url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000100140001&fltDispCatNo=&prdSort=01&pageIdx={page}"
        driver.get(url)

        time.sleep(1.0)
        prd_list = driver.find_element(By.ID,"Contents")
        # prd = prd_list.find_element(By.CLASS_NAME,"cate_prd_list")      # 한 페이지에 이 클래스 이름을 가진 ul이 여러 개고 그 ul마다 각각 상품 네개씩 있음
        prd = prd_list.find_elements(By.CLASS_NAME,"prd_info")              # 그냥 바로 상품 각각 찾기

        for i in range(len(prd)):
            prds = driver.find_elements(By.CLASS_NAME, "prd_info")  # Refresh the list of prd elements
            each_prd = prds[i]
            each_prd.click()

            info_btn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "buyInfo")))
            info_btn.click()

            prd_items = driver.find_element(By.CLASS_NAME, "right_area")
            prd_brands.append(prd_items.find_element(By.ID, "moveBrandShop").text)    
            prd_name = prd_items.find_element(By.CLASS_NAME, "prd_name").text
            prd_names.append(prd_name)
            prd_prices.append(prd_items.find_element(By.CLASS_NAME, "price").text.split('\n')[0])
            
            
            prd_images_area = driver.find_element(By.ID, "prd_thumb_list")
            thumbs = prd_images_area.find_elements(By.TAG_NAME, "li")
            imageNames = []
            for idx, li in enumerate(thumbs):
                
                imgs = li.find_element(By.TAG_NAME, "img")
                imgsrc = imgs.get_attribute("src")
                imgres = requests.get(imgsrc)

                imageName = f"./prdimgs/page_{page}_prd_{i}_thumb{idx + 1}.jpg"
                with open(imageName, "wb") as f:
                    f.write(imgres.content)
                imageNames.append(imageName)
                
            prd_thumbs.append(imageNames)
            prd_info = WebDriverWait(driver,10).until(EC.visibility_of_element_located((By.ID, "artcInfo")))
            prd_infos.append(prd_info.text.split("\n"))
            driver.back()
            time.sleep(0.2)
finally:
    driver.quit()

In [55]:
import pandas as pd
serum_new = pd.DataFrame({"제품명":prd_names, "브랜드":prd_brands, "가격":prd_prices,'상품정보':prd_infos,'썸네일':prd_thumbs})
new_name = f'./output/serum_items_{lastpage}.csv'
serum_new.to_csv(new_name,index=False)
serum_new

,제품명,브랜드,가격,상품정보,썸네일
0,[리쥬란 독자소재_저자극 안심성분]리쥬란 더마 힐러 트리트먼트 에센스 70g 기획,리쥬란,"40,000","[내용물의 용량 또는 중량 70g, 제품 주요 사양 모든피부용, 사용기한(또는 개봉...","[./prdimgs/page_21_prd_0_thumb1.jpg, ./prdimgs..."
1,헤라 옴므 파워부스팅 페이스 트리트먼트 110ML,헤라,"75,000","[내용물의 용량 또는 중량 110ML, 제품 주요 사양 모든 피부타입, 사용기한(또...","[./prdimgs/page_21_prd_1_thumb1.jpg, ./prdimgs..."
2,토리든 밸런스풀 시카 진정 세럼 50ml,토리든,"23,000","[내용물의 용량 또는 중량 50ml, 제품 주요 사양 모든피부타입, 사용기한(또는 ...","[./prdimgs/page_21_prd_2_thumb1.jpg, ./prdimgs..."
3,에스티 로더 마이크로 에센스 트리트먼트 로션 프레시 100ml,에스티로더,"95,000","[내용물의 용량 또는 중량 100ml, 제품 주요 사양 모든 피부 타입, 사용기한(...",[./prdimgs/page_21_prd_3_thumb1.jpg]
4,에스티 로더 마이크로 에센스 트리트먼트 로션 100ml,에스티로더,"95,000","[내용물의 용량 또는 중량 100ml, 제품 주요 사양 모든 피부 타입, 사용기한(...",[./prdimgs/page_21_prd_4_thumb1.jpg]
...,...,...,...,...,...
76,유리아쥬 배리어덤 시카 데일리 세럼 30ml 기획 (+시카 토너50ml 증정),유리아쥬,"44,000",[내용물의 용량 또는 중량 [배리어덤 시카 데일리 세럼]30ml [시카토너]50ml...,"[./prdimgs/page_24_prd_4_thumb1.jpg, ./prdimgs..."
77,유리아쥬 이제악 뉴스킨 세럼40ml 기획 (+이제악 클렌징젤50ml 증정),유리아쥬,"30,000",[내용물의 용량 또는 중량 [이제악 뉴스킨세럼] 40ml [이제악 클렌징젤] 50m...,"[./prdimgs/page_24_prd_5_thumb1.jpg, ./prdimgs..."
78,차앤박(CNP) 비타비 브라이트닝 앰플 15ml,차앤박,"29,000","[내용물의 용량 또는 중량 차앤박 비타비 브라이트닝 앰플 15ml (2022), 제...","[./prdimgs/page_24_prd_6_thumb1.jpg, ./prdimgs..."
79,닥터지 로얄 블랙 스네일 퍼스트 에센스 165ml 기획 (+화장솜),닥터지,"38,000",[내용물의 용량 또는 중량 로얄 블랙스네일 퍼스트 에센스 165ml + 화장솜 증정...,"[./prdimgs/page_24_prd_7_thumb1.jpg, ./prdimgs..."


In [57]:
final_new = pd.read_csv('./output/serum_items_all_to_21.csv')
final_new = pd.concat([final_new,serum_new], axis=0, ignore_index=True)
final_new

,제품명,브랜드,가격,상품정보,썸네일
0,[아랑Pick] VT 리들샷 100 에센스 50ml,브이티,"32,000","['내용물의 용량 또는 중량 50ml', '제품 주요 사양 모든피부 타입', '사용...","['./prdimgs/page_1_prd_0_thumb1.jpg', './prdim..."
1,[제이나PICK / 1월 올영픽] 아비브 부활초 비피다 세럼 퍼밍 드롭 50ml 기...,아비브,"32,000","['내용물의 용량 또는 중량 50ml + 30ml', '제품 주요 사양 모든 피부용...","['./prdimgs/page_1_prd_1_thumb1.jpg', './prdim..."
2,[피부결] 넘버즈인 3번 보들보들 결 세럼 80ml 기획(+3번 결 세럼팩),넘버즈인,"34,000","['내용물의 용량 또는 중량 80ml', '제품 주요 사양 모든 피부 용', '사용...","['./prdimgs/page_1_prd_2_thumb1.jpg', './prdim..."
3,[2023어워즈] 구달 청귤 비타C 잡티케어 세럼 50ml 어워즈 한정기획 (50m...,구달,"43,000","['내용물의 용량 또는 중량 본품 50ml+50ml 리필파우치', '제품 주요 사양...","['./prdimgs/page_1_prd_3_thumb1.jpg', './prdim..."
4,[형원PICK] 웰라쥬 리얼 히알루로닉 블루 100 앰플 75ml 1+1 기획,웰라쥬,"46,000","['내용물의 용량 또는 중량 75ml+75ml', '제품 주요 사양 모든 피부용',...","['./prdimgs/page_1_prd_4_thumb1.jpg', './prdim..."
...,...,...,...,...,...
580,유리아쥬 배리어덤 시카 데일리 세럼 30ml 기획 (+시카 토너50ml 증정),유리아쥬,"44,000",[내용물의 용량 또는 중량 [배리어덤 시카 데일리 세럼]30ml [시카토너]50ml...,"[./prdimgs/page_24_prd_4_thumb1.jpg, ./prdimgs..."
581,유리아쥬 이제악 뉴스킨 세럼40ml 기획 (+이제악 클렌징젤50ml 증정),유리아쥬,"30,000",[내용물의 용량 또는 중량 [이제악 뉴스킨세럼] 40ml [이제악 클렌징젤] 50m...,"[./prdimgs/page_24_prd_5_thumb1.jpg, ./prdimgs..."
582,차앤박(CNP) 비타비 브라이트닝 앰플 15ml,차앤박,"29,000","[내용물의 용량 또는 중량 차앤박 비타비 브라이트닝 앰플 15ml (2022), 제...","[./prdimgs/page_24_prd_6_thumb1.jpg, ./prdimgs..."
583,닥터지 로얄 블랙 스네일 퍼스트 에센스 165ml 기획 (+화장솜),닥터지,"38,000",[내용물의 용량 또는 중량 로얄 블랙스네일 퍼스트 에센스 165ml + 화장솜 증정...,"[./prdimgs/page_24_prd_7_thumb1.jpg, ./prdimgs..."


In [58]:
final_new.to_csv(f'./output/serum_items_all_to_{lastpage}.csv',index=False)

In [53]:
24*lastpage

504